# 実践演習 Day 1：streamlit と FastAPI のデモ

このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Face からモデルを用いた Streamlit のデモアプリ
- FastAPI と ngrok を使用した API の公開方法

演習を始める前に、HuggingFace と ngrok のアカウントを作成し、
それぞれの API トークンを取得する必要があります。

演習の時間では、以下の 3 つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2 つ目や 3 つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項

「02_streamlit_app」と「03_FastAPI」では、GPU を使用します。

これらを実行する際は、Google Colab 画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---


# 環境変数の設定（1~3 共有）


「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。


In [1]:
from dotenv import load_dotenv, find_dotenv

%cd ~/workspace/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

/home/saito/workspace/lecture-ai-engineering/day1


/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


True

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。


In [3]:
%cd 01_streamlit_UI

/home/saito/workspace/lecture-ai-engineering/day1/01_streamlit_UI


/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


必要なライブラリをインストールします。


In [4]:
%%capture
!pip install -r requirements.txt

ngrok のトークンを使用して、認証を行います。


In [5]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /home/saito/.config/ngrok/ngrok.yml                          


アプリを起動します。


In [6]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://bfdd-133-15-57-27.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://133.15.57.27:8501
  External URL: http://133.15.57.27:8501

^C
  Stopping...


t=2025-04-23T18:21:27+0900 lvl=warn msg="failed to open private leg" id=e2e7299a8f4f privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:21:27+0900 lvl=warn msg="failed to open private leg" id=3e81fbad28fb privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:21:29+0900 lvl=warn msg="failed to open private leg" id=5556051f660c privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:21:29+0900 lvl=warn msg="failed to open private leg" id=b8d10844083d privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:21:30+0900 lvl=warn msg="failed to open private leg" id=d3b9ad8e15b6 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:21:30+0900 lvl=warn msg="failed to open private leg" id=5b713f7b2a90 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection r

公開 URL の後に記載されている URL にブラウザでアクセスすると、streamlit の UI が表示されます。

app.py のコメントアウトされている箇所を編集することで、UI がどの様に変化するか確認してみましょう。

streamlit の公式ページには、ギャラリーページがあります。

streamlit を使うと python という一つの言語であっても、様々な UI を実現できることがわかると思います。

https://streamlit.io/gallery


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [7]:
from pyngrok import ngrok

ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。


In [2]:
%cd ~/workspace/lecture-ai-engineering/day1/02_streamlit_app

/home/saito/workspace/lecture-ai-engineering/day1/02_streamlit_app


ngrok と huggigface のトークンを使用して、認証を行います。


In [3]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /home/saito/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/saito/.cache/huggingface/token
Login successful


stramlit で Huggingface のトークン情報を扱うために、streamlit 用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。


In [4]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs(".streamlit", exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {"huggingface": {"token": os.environ.get("HUGGINGFACE_TOKEN", "")}}

# 設定ファイルを書き込む
with open(".streamlit/secrets.toml", "w") as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_app では、Huggingface からモデルをダウンロードするため、初回起動には 2 分程度時間がかかります。

この待ち時間を利用して、app.py のコードを確認してみましょう。


In [8]:
# アプリ起動セル (ID: TBQyTTWTELSP) - 修正後
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://6097-133-15-57-27.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://133.15.57.27:8501
  External URL: http://133.15.57.27:8501

NLTK loaded successfully.
[2025-04-23 18:43:37,255] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
2025-04-23 18:43:38.104 Uncaught app execution
Traceback (most recent call last):
  File "/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/import_utils.py", line 1967, in _get_module
    retur

t=2025-04-23T18:44:50+0900 lvl=warn msg="failed to open private leg" id=f11f522119d2 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:44:50+0900 lvl=warn msg="failed to open private leg" id=d9b0ee789769 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:44:52+0900 lvl=warn msg="failed to open private leg" id=5a5530051dac privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:44:52+0900 lvl=warn msg="failed to open private leg" id=dbf338f3f6ef privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:44:53+0900 lvl=warn msg="failed to open private leg" id=5830821bd8d0 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection refused"
t=2025-04-23T18:44:54+0900 lvl=warn msg="failed to open private leg" id=1fdb589ed008 privaddr=localhost:8501 err="dial tcp 127.0.0.1:8501: connect: connection r

アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、Streamlit による UI 部分だけではなく、SQLite を使用したチャット履歴の保存や LLM のモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理の UI を提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションの UI ロジックを管理します。
- **`llm.py`**: LLM モデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLite データベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEU スコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要な Python パッケージ。


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [6]:
from pyngrok import ngrok

ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。


In [10]:
%cd ~/workspace/lecture-ai-engineering/day1/03_FastAPI

/home/saito/workspace/lecture-ai-engineering/day1/03_FastAPI


/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/saito/.pyenv/versions/3.12.0/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


必要なライブラリをインストールします。


In [11]:
%%capture
!pip install -r requirements.txt

ngrok と huggigface のトークンを使用して、認証を行います。


In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。


In [ ]:
!python app.py

FastAPI が起動すると、API とクライアントが通信するための URL（エンドポイント）が作られます。

URL が作られるのと合わせて、Swagger UI という Web インターフェースが作られます。

Swagger UI にアクセスすることで、API の仕様を確認できたり、API をテストすることができます。

Swagger UI を利用することで、API を通して LLM を動かしてみましょう。


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [13]:
from pyngrok import ngrok

ngrok.kill()